In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv("logs_dataset.csv")

In [3]:
df.shape

(721547, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721547 entries, 0 to 721546
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   @timestamp  721547 non-null  object
 1   _id         721547 non-null  object
 2   ip_address  721547 non-null  object
dtypes: object(3)
memory usage: 16.5+ MB


In [5]:
df.head()

,@timestamp,_id,ip_address
0,"July 8th 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8th 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8th 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8th 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8th 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60


In [6]:
# Unique IP Address

print("The number of unqiue IP Addresses are: ",df['ip_address'].nunique())

The number of unqiue IP Addresses are:  386


In [7]:
df['@timestamp'] = (
    df['@timestamp']
    .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
)
df['@timestamp'] = pd.to_datetime(
    df['@timestamp'],
    format='%B %d %Y, %H:%M:%S.%f'
)

In [8]:
df.sort_values(['ip_address', '@timestamp'], inplace=True)

In [9]:
df.head()

,@timestamp,_id,ip_address
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1


In [10]:
df['shift_time'] = df.groupby(['ip_address'])['@timestamp'].shift(1)

In [11]:
df.head()

,@timestamp,_id,ip_address,shift_time
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45


In [12]:
df['time_diff'] = (df['@timestamp']-df['shift_time']).dt.seconds//60

In [13]:
df['date'] = df['@timestamp'].dt.date

In [14]:
df['weekday'] = df['@timestamp'].dt.weekday

In [15]:
df['hour'] = df['@timestamp'].dt.hour

In [16]:
df['is_weekend'] = ((df['weekday'] == 5) | (df['weekday'] == 6)).astype(int)

In [17]:
df.head()

,@timestamp,_id,ip_address,shift_time,time_diff,date,weekday,hour,is_weekend
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT,NaN,2019-06-09,6,0,1
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09,82.0,2019-06-09,6,1,1
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39,104.0,2019-06-09,6,3,1
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49,0.0,2019-06-09,6,3,1
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45,28.0,2019-06-09,6,3,1


In [18]:
df['hour_bucket'] = df['hour']//4
# This integer division by 4, so we can analyse 4-hour buckets instead of every single hour

In [19]:
ip_addr ='ip_address'

In [20]:
ip_counts = df.groupby(ip_addr)['@timestamp'].count().reset_index()

In [21]:
ip_counts.head()

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


In [22]:
ip_counts = ip_counts.rename(columns={'@timestamp':'total_count'})

In [23]:
daily_counts = df.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()

In [24]:
daily_counts = daily_counts.rename(columns={'@timestamp':'daily_counts'})

In [25]:
daily_counts_avg = daily_counts.groupby(ip_addr).daily_counts.median().reset_index()

In [26]:
daily_counts_avg.head(5)

,ip_address,daily_counts
0,10.1.1.1,40.0
1,10.1.1.100,78.0
2,10.1.1.101,40.0
3,10.1.1.106,35.5
4,10.1.1.109,42.5


In [27]:
weekend_counts = df.groupby([ip_addr, 'is_weekend'])['@timestamp'].count().reset_index()

In [28]:
weekend_counts

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
...,...,...,...
767,10.1.2.90,1,871
768,10.1.2.95,0,1973
769,10.1.2.95,1,895
770,10.1.2.99,0,978
